**Installation**

In [ ]:
!pip install langchain
!pip install langchain-community
!pip install langchain-groq
!pip install pypdf
!pip install unstructured[md]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 24.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

**Groq API key**

In [ ]:
groq_api_key = "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"

**Import Section**

In [ ]:
import os
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.document_loaders import PyPDFLoader
from langchain_groq import ChatGroq
import nest_asyncio  # noqa: E402

nest_asyncio.apply()

**Initialize the chat model**




In [ ]:

chat_model = ChatGroq(
    temperature=0,
    model_name="llama3-8b-8192",
    api_key=groq_api_key,
)

Write a Custome prompt

In [ ]:
# Define custom prompt template
custom_prompt_template = """You are an assistant. Please provide relevant answers to questions based on the provided context. If you don't know the answer, just say that you don't know; don't try to make up an answer.

Context: {context}

Question: {question}

Helpful answer:
"""

**Function to load and split PDF documents**

In [ ]:
def load_and_prepare_documents(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load_and_split()
    return "\n".join([doc.page_content for doc in documents])

# Load documents from PDF
context = load_and_prepare_documents("/content/invoice_107_charspace_108.pdf")

**Function to create a prompt template**

In [ ]:
def create_prompt_template(template_str):
    return PromptTemplate(template=template_str, input_variables=['context', 'question'])

prompt = create_prompt_template(custom_prompt_template)

In [ ]:
def generate_output(context, question):
  # Create the LLMChain
  llm_chain = LLMChain(
      llm=chat_model,
      prompt=prompt
  )
  # Execute the chain with the question and context
  response = llm_chain.run({
      "context": context,
      "question": question
  })
  # Print the response
  print(question)
  print(response)
  print("******************")

In [ ]:
questions = ["What is invoice no?",
             "What is Date mentioned in invoice?",
             "What is Client Name?",
             "What is Seller Name?",
             "What is Client Address?",
             "What is Seller Address?",
             "What is Client Tax Id?",
             "What is Seller Tax Id?",
             "What is IBAN No?"]

for question in questions:
  generate_output(context, question)





/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


What is invoice no?
The invoice number is 82545881.
******************
What is Date mentioned in invoice?
The date mentioned in the invoice is 09/25/2011.
******************
What is Client Name?
The client name is Keller-Crosby.
******************
What is Seller Name?
The seller's name is Campbell, Callahan and Gomez.
******************
What is Client Address?
The client address is:

280 Kim Valleys Suite 217
Angelaburgh, DE 97356
******************
What is Seller Address?
The seller's address is:

2969 Todd Orchard Apt. 721
Port James, FL 83598
******************
What is Client Tax Id?
The Client Tax Id is 941-79-6209.
******************
What is Seller Tax Id?
The Seller's Tax ID is 958-83-8233.
******************
What is IBAN No?
The IBAN number is GB86WKRJ04578791818338.
******************
